In [19]:
from dash import Dash, dcc, html, Input, Output
import pandas as pd
import plotly.express as px
import webbrowser
from threading import Timer
import socket

df = pd.read_csv('project_1_python.csv')


df['date'] = pd.to_datetime(df['date'])
df.sort_values(['location', 'date'], inplace=True)
df['cumulative_cases'] = df.groupby('location')['new_cases'].cumsum()
df['cumulative_deaths'] = df.groupby('location')['new_deaths'].cumsum()
app = Dash(__name__)

app.layout = html.Div([
    html.H1('COVID-19 Data Visualization', style={'textAlign': 'center'}),
    dcc.Dropdown(
        id='country-selector',
        options=[{'label': i, 'value': i} for i in df['location'].unique()],
        value='United States',  
        style={'width': '50%', 'margin': 'auto'}
    ),
    html.Div([
        dcc.Graph(id='cases-graph'),
        dcc.Graph(id='deaths-graph')
    ], style={'display': 'flex', 'justifyContent': 'space-around'}),
])


@app.callback(
    [Output('cases-graph', 'figure'),
     Output('deaths-graph', 'figure')],
    [Input('country-selector', 'value')]
)
def update_graphs(selected_country):
    if selected_country not in df['location'].unique():
        return {}, {} 

    filtered_data = df[df['location'] == selected_country]

    if filtered_data.empty:
        return {}, {} 
    fig_cases = px.line(
        filtered_data, 
        x='date', 
        y='cumulative_cases', 
        title=f'Cumulative COVID-19 Cases in {selected_country}'
    )
    fig_deaths = px.line(
        filtered_data, 
        x='date', 
        y='cumulative_deaths', 
        title=f'Cumulative COVID-19 Deaths in {selected_country}'
    )
    return fig_cases, fig_deaths

def find_free_port():
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(("", 0))
    port = s.getsockname()[1]
    s.close()
    return port


def open_browser(port):
    webbrowser.open_new(f"http://127.0.0.1:{port}/")

if __name__ == '__main__':
    port = find_free_port() 
    Timer(1, lambda: open_browser(port)).start()  
    app.run_server(port=port, host='127.0.0.1', debug=True, use_reloader=False)





In [20]:
from dash import Dash, dcc, html, Input, Output
import pandas as pd
import plotly.express as px
import webbrowser
import socket
from threading import Timer

# Assume df is already loaded and prepared
prepared_df = df[df['date'] == df['date'].max()].fillna(0)

metrics = {
    'total_cases': 'Total Cases',
    'total_deaths': 'Total Deaths',
    'total_tests': 'Total Tests',
    'total_vaccinations': 'Total Vaccinations',
    'fully_vaccinated': 'Fully Vaccinated Individuals'
}

continents_list = list(prepared_df['continent'].drop_duplicates())

covid_app = Dash(__name__)
covid_app.layout = html.Div([
    html.H1('Global COVID-19 Dashboard', style={'textAlign': 'center'}),
    dcc.Dropdown(
        id='continent-dropdown',
        options=[{'label': cont, 'value': cont} for cont in continents_list],
        value=continents_list[0]
    ),
    dcc.Dropdown(
        id='metric-dropdown',
        options=[{'label': desc, 'value': met} for met, desc in metrics.items()],
        value='total_cases'
    ),
    dcc.Graph(id='global-map')
])

@covid_app.callback(
    Output('global-map', 'figure'),
    [Input('continent-dropdown', 'value'), Input('metric-dropdown', 'value')]
)
def update_map(selected_continent, selected_metric):
    data_subset = prepared_df[prepared_df['continent'] == selected_continent]
    fig = px.scatter_mapbox(
        data_frame=data_subset,
        lat='latitude',
        lon='longitude',
        size=selected_metric,
        color=selected_metric,
        size_max=30,
        hover_data={'location': True},
        color_continuous_scale='Blues',
        mapbox_style='light',
        zoom=2,
        height=500,
        title=f'COVID-19 - {metrics[selected_metric]} in {selected_continent}'
    )
    fig.update_layout(mapbox_accesstoken='pk.eyJ1IjoiYmFrc2lvdmEiLCJhIjoiY2x2dGJ0a3JtMWJyNzJycnJxemFidXI4bCJ9.KeKGQiI2SwpHrN83kzB6Rg')
    return fig

def find_free_port():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(("", 0))
        return s.getsockname()[1]

def open_browser(port):
    webbrowser.open_new(f"http://127.0.0.1:{port}/")

if __name__ == '__main__':
    port = find_free_port() 
    Timer(1, lambda: open_browser(port)).start()
    covid_app.run_server(port=port, debug=True, use_reloader=False)


In [23]:
from dash import Dash, dcc, html, Input, Output
import pandas as pd
import plotly.express as px
import webbrowser
from threading import Timer
import socket

# Load your data
df = pd.read_csv('project_1_python.csv')
prepared_df = df[df['date'] == df['date'].max()].fillna(0)

metrics = {
    'total_cases': 'Total Cases',
    'total_deaths': 'Total Deaths',
    'total_tests': 'Total Tests',
    'total_vaccinations': 'Total Vaccinations',
    'fully_vaccinated': 'Fully Vaccinated Individuals'
}

continents_list = list(prepared_df['continent'].drop_duplicates())

covid_app = Dash(__name__)
covid_app.layout = html.Div([
    html.H1('Global COVID-19 Dashboard', style={'textAlign': 'center'}),
    dcc.Dropdown(
        id='continent-dropdown',
        options=[{'label': cont, 'value': cont} for cont in continents_list],
        value=continents_list[0]
    ),
    dcc.Dropdown(
        id='metric-dropdown',
        options=[{'label': desc, 'value': met} for met, desc in metrics.items()],
        value='total_cases'
    ),
    dcc.Graph(id='global-map')
])

@covid_app.callback(
    Output('global-map', 'figure'),
    [Input('continent-dropdown', 'value'), Input('metric-dropdown', 'value')]
)
def update_map(selected_continent, selected_metric):
    data = prepared_df[prepared_df['continent'] == selected_continent]
    if data.empty:
        return {}
    fig = px.scatter_mapbox(
        data=data,
        lat='latitude',
        lon='longitude',
        size=selected_metric,
        color=selected_metric,
        size_max=30,
        hover_data={'location': True},
        color_continuous_scale='Blues',
        mapbox_style='light',
        zoom=2,
        height=600,
        title=f'COVID-19 - {metrics[selected_metric]} in {selected_continent}'
    )
    fig.update_layout(mapbox_accesstoken='pk.eyJ1IjoiYmFrc2lvdmEiLCJhIjoiY2x2dGJ0a3JtMWJyNzJycnJxemFidXI4bCJ9.KeKGQiI2SwpHrN83kzB6Rg')
    return fig

# Function to find a free port
def find_free_port():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(("", 0))
        return s.getsockname()[1]

# Function to open the app in a web browser
def launch_browser(port):
    webbrowser.open_new(f"http://127.0.0.1:{port}/")

# Running the app
if __name__ == '__main__':
    port = find_free_port()
    Timer(1, lambda: launch_browser(port)).start()
    covid_app.run_server(port=port, debug=True, use_reloader=False)


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[23], line 46, in update_map(
    selected_continent='Asia',
    selected_metric='total_cases'
)
     44 if data.empty:
     45     return {}
---> 46 fig = px.scatter_mapbox(
        px = <module 'plotly.express' from '/Users/katka/anaconda3/lib/python3.11/site-packages/plotly/express/__init__.py'>
        data =          index iso_code continent              location        date  \
845        845      AFG      Asia           Afghanistan  2022-06-18   
7603      7603      ARM      Asia               Armenia  2022-06-18   
10991    10991      AZE      Asia            Azerbaijan  2022-06-18   
12662    12662      BHR      Asia               Bahrain  2022-06-18   
13495    13495      BGD      Asia            Bangladesh  2022-06-18   
19327    19327      BTN      Asia                Bhutan  2022-06-18   
24293    25954      BRN      

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[23], line 46, in update_map(
    selected_continent='Asia',
    selected_metric='total_cases'
)
     44 if data.empty:
     45     return {}
---> 46 fig = px.scatter_mapbox(
        px = <module 'plotly.express' from '/Users/katka/anaconda3/lib/python3.11/site-packages/plotly/express/__init__.py'>
        data =          index iso_code continent              location        date  \
845        845      AFG      Asia           Afghanistan  2022-06-18   
7603      7603      ARM      Asia               Armenia  2022-06-18   
10991    10991      AZE      Asia            Azerbaijan  2022-06-18   
12662    12662      BHR      Asia               Bahrain  2022-06-18   
13495    13495      BGD      Asia            Bangladesh  2022-06-18   
19327    19327      BTN      Asia                Bhutan  2022-06-18   
24293    25954      BRN      